In [41]:
import json
import csv
import pandas as pd
import sqlite3
import networkx as nx

In [89]:
ls

Collection JSONs/                   nft_database_simple.db
all_transactions.csv                nft_database_simple_new.db
generate_top_pr_edges.sql           nft_database_simple_new.db-journal
json_to_sqlite(not working).ipynb   page_rank_files/
json_to_sqlite(working).ipynb       terraform.csv
network analysis.ipynb              union_tables.sql
nft_database.db


In [270]:
conn = sqlite3.connect('nft_database_simple_new.db')
c = conn.cursor()

In [281]:
collection = 'all'
collectionWith = '"' + collection + '"'
weight = 'weight'

In [272]:
df_all_transactions = pd.read_sql_query('select from_address, to_address, ' + weight + ' from all_transactions_weighted where collection_name = ' + collectionWith, conn)
# df_all_transactions = pd.read_sql_query('select from_address, to_address, ' + weight + ' from all_transactions_weighted', conn)
df_edge = df_all_transactions[['from_address', 'to_address', weight]]
df_edge.head()

from_address  \
0  0xa53496b67eec749ac41b4666d63228a0fb0409cf   
1  0xd73e0def01246b650d8a367a4b209be59c8be8ab   
2  0xcbd482af76059e2a5e4a825c4cd5ced9f2dfe286   
3  0xf6764f702650eb66b2bd7368e2fef7133a8f4f80   
4  0x718a085f6d45a955395881bceff30ec97ffb1013   

                                   to_address    weight  
0  0xd73e0def01246b650d8a367a4b209be59c8be8ab  1.000000  
1  0xa53496b67eec749ac41b4666d63228a0fb0409cf  0.998597  
2  0x577ebc5de943e35cdf9ecb5bbe1f7d7cb6c7c647  0.490182  
3  0x718a085f6d45a955395881bceff30ec97ffb1013  0.422160  
4  0xf6764f702650eb66b2bd7368e2fef7133a8f4f80  0.421459

In [273]:
query_wallets = 'select from_address from all_transactions where collection_name = ' + collectionWith + ' union select to_address from all_transactions where collection_name = ' + collectionWith
# query_wallets = 'select from_address from all_transactions union select to_address from all_transactions'
df_wallets = pd.read_sql_query(query_wallets, conn)
df_wallets.head()

from_address
0  0x0000000000000000000000000000000000000000
1  0x0000000000000000000000000000000000000001
2  0x000000000000000000000000000000000000dead
3  0x00000000000000000000005cda7ec9514b4f5959
4  0x0000000000015b23c7e20b0ea5ebd84c39dcbe60

In [274]:
G = nx.from_pandas_edgelist(df=df_edge, source='from_address', target='to_address', edge_attr=[weight])

In [275]:
G.add_nodes_from(nodes_for_adding=df_wallets.from_address.tolist())

In [276]:
df_edges = pd.DataFrame(G.edges.data())
df_edges = df_edges.rename(columns={0: "source", 1: "target", 2: "attr"})
df_edges.head(15)

source  \
0   0xa53496b67eec749ac41b4666d63228a0fb0409cf   
1   0xa53496b67eec749ac41b4666d63228a0fb0409cf   
2   0xd73e0def01246b650d8a367a4b209be59c8be8ab   
3   0xcbd482af76059e2a5e4a825c4cd5ced9f2dfe286   
4   0xcbd482af76059e2a5e4a825c4cd5ced9f2dfe286   
5   0xcbd482af76059e2a5e4a825c4cd5ced9f2dfe286   
6   0xcbd482af76059e2a5e4a825c4cd5ced9f2dfe286   
7   0xcbd482af76059e2a5e4a825c4cd5ced9f2dfe286   
8   0xcbd482af76059e2a5e4a825c4cd5ced9f2dfe286   
9   0xcbd482af76059e2a5e4a825c4cd5ced9f2dfe286   
10  0xcbd482af76059e2a5e4a825c4cd5ced9f2dfe286   
11  0xcbd482af76059e2a5e4a825c4cd5ced9f2dfe286   
12  0xcbd482af76059e2a5e4a825c4cd5ced9f2dfe286   
13  0xcbd482af76059e2a5e4a825c4cd5ced9f2dfe286   
14  0xcbd482af76059e2a5e4a825c4cd5ced9f2dfe286   

                                        target  \
0   0xd73e0def01246b650d8a367a4b209be59c8be8ab   
1   0x6e82d57018bc95ef6d8b82bdc42449f77941dc32   
2   0x3bfe8b28de68aba3723b801f5d0f0d10a701293c   
3   0x577ebc5de943e35cdf9ecb5bbe1f7d7cb6c7c647   
4   0x7174039818a41e1ae40fdcfa3e293b0f41592af2   
5   0x7760e0243ca9baa630412865df7b39afba42ff0f   
6   0x5f7190b7943858b7bdd24f069241612091c6a986   
7   0x00bd3a6660309fb9e0129b9b777a9ccb9c2869dc   
8   0x0a690b298f84d12414f5c8db7de1ece5a4605877   
9   0x28b564f44a13f3fa5ef9695116822c89c9cb2a5a   
10  0x6611fe71c233e4e7510b2795c242c9a57790b376   
11  0x5b098b00621eda6a96b7a476220661ad265f083f   
12  0x0544fbed9b72aa036517b21d1db50201a17d09ce   
13  0x2b88d3a67b6f028fdca7fab5a0f80b85775c89cb   
14  0x2e5d4bafaa4d2a5476ee423adbc7f5863e452f12   

                                 attr  
0      {'weight': 0.9985974754558204}  
1    {'weight': 0.001402524544179523}  
2   {'weight': 0.0007012622720897616}  
3      {'weight': 0.4901823281907433}  
4    {'weight': 0.006311360448807854}  
5    {'weight': 0.004207573632538569}  
6    {'weight': 0.002805049088359046}  
7    {'weight': 0.001402524544179523}  
8    {'weight': 0.001402524544179523}  
9    {'weight': 0.001402524544179523}  
10   {'weight': 0.001402524544179523}  
11  {'weight': 0.0007012622720897616}  
12  {'weight': 0.0007012622720897616}  
13  {'weight': 0.0007012622720897616}  
14  {'weight': 0.0007012622720897616}

In [253]:
# Neighbors of a node
# pd.DataFrame(list(G.neighbors('0x000000000000000000000000000000000000dead')))

In [277]:
pr = nx.pagerank(G, weight=weight)
list = [(k, v) for k, v in pr.items()]
df_pr = pd.DataFrame(list).rename(columns={0:'wallet_hash', 1:'page_rank'})
df_pr = df_pr.sort_values(by=['page_rank'], ascending=False)
df_pr.head(50)

wallet_hash  page_rank
129    0x620b70123fb810f6c653da7644b5dd0b6312e4d8   0.005817
377    0xc310e760778ecbca4c65b6c559874757a4c4ece0   0.004051
47     0xd387a6e4e84a6c86bd90c158c6028a58cc8ac459   0.002916
38     0xb7f7f6c52f2e2fdb1963eab30438024864c313f6   0.002711
1089   0xe052113bd7d7700d623414a0a4585bcae754e9d5   0.000918
27     0xdcf809bba1d9ee75c6991f1e2974163cf2555418   0.000868
391    0x2d1194e75b408c9395a8bbbedc0079322b62c0a4   0.000816
31     0x1919db36ca2fa2e15f9000fd9cdc2edcf863e685   0.000757
1634   0x83c8f28c26bf6aaca652df1dbbe0e1b56f8baba2   0.000754
39     0x53ede7cae3eb6a7d11429fe589c0278c9acbe21a   0.000607
36     0x88341d1a8f672d2780c8dc725902aae72f143b0c   0.000606
527    0x020ca66c30bec2c4fe3861a94e4db4a498a35872   0.000573
6      0xfc840af78a496b52e378dfa94838402e1b7f71dd   0.000532
956    0x113d754ff2e6ca9fd6ab51932493e4f9dabdf596   0.000502
3      0x577ebc5de943e35cdf9ecb5bbe1f7d7cb6c7c647   0.000502
198    0x38a4d889a1979133fbc1d58f970f0953e3715c26   0.000495
183    0x721931508df2764fd4f70c53da646cb8aed16ace   0.000457
284    0x114f1388fab456c4ba31b1850b244eedcd024136   0.000448
10845  0x265cdd11b3abb0432a35300f6afb214777280817   0.000441
302    0x54b174179ae825ed630da40b625bb3c883cd40ae   0.000438
332    0xeb11f192e91e2d4abff199b58222c89472584bf4   0.000437
474    0x49c3feafddaefc3bed06f4ff87ce86610c2c1076   0.000421
89     0xaf93fcce0548d3124a5fc3045adaf1dde4e8bf7e   0.000410
353    0x1434a8eaa6d3725d44984d674e4f2dd9cc472bf4   0.000406
786    0xea39c551834d07ee2ee87f1ceff843c308e089af   0.000403
14     0x00d7c902fbbcd3c9db2da80a439c94486c50eb81   0.000400
1614   0x53fed1eb683745352886a363cc0b6e3415824ec8   0.000388
86     0x1999a92711cf8815b7f6b07bd34b4287d853cfba   0.000382
74     0x46e449a3f88d0e35b4520bc36e8dfda195c896b0   0.000373
28     0xe72eb31b59f85b19499a0f3b3260011894fa0d65   0.000359
4542   0x6f4a2d3a4f47f9c647d86c929755593911ee91ec   0.000358
4199   0xbec69dfce4c1fa8b7843fee1ca85788d84a86b06   0.000353
5805   0x6903f3a1540946c34e3a577ad6ffcbe43356dcbe   0.000353
4440   0x3c6137504c38215fea30605b3e364a23c1d3e14f   0.000351
6150   0xbd74c3f96c38478460a0e3c88ac86dd133af72be   0.000342
511    0x268d3a8c4701000546dd0bf959bf6c8956cc3e5e   0.000341
3326   0x65ab793bd82cf8d9f035d4742b95e2c16a6b8849   0.000339
1182   0xb3ee5011a7965905cde351ea4905ff4725189a3b   0.000336
2919   0xe18a94a1e60903b1ec39d9104bed27d9a73ad9e2   0.000329
151    0x72fae93d08a060a7f0a8919708c0db74ca46cbb6   0.000319
164    0x62bdc64a14405e797f86796ea3c2476209ee0bf9   0.000317
9995   0x904778b44bd12a1f7c3f1b95ff3381a952296cd1   0.000313
1542   0xb697b9455eeee9574ec5c5350cb10c8a5b752a39   0.000306
506    0x2dab4ce3490bb50b2ea4c07ab1b6a9cfe29d89b3   0.000305
4549   0xb32b4350c25141e779d392c1dbe857b62b60b4c9   0.000297
737    0x80845058350b8c3df5c3015d8a717d64b3bf9267   0.000296
790    0x4fed1f42ba007d1f7a6f0df4ae100170adb305c6   0.000291
2729   0x0e9aed5c7721c642a032812c2c4816f7d6cb87d7   0.000288
955    0x1731bdd386e021a7744282454887a87551725819   0.000283
10     0xc352b534e8b987e036a93539fd6897f53488e56a   0.000280

In [278]:
cc = nx.clustering(G)
df_cc = pd.DataFrame([(k, v) for k, v in cc.items()]).rename(columns={0:'wallet_hash', 1:'clustering coefficient'})
df_cc = df_cc.sort_values(by=['clustering coefficient'], ascending=False)
df_cc

wallet_hash  clustering coefficient
54152   0x21b0aa981db32ae49a92bc1e06f048b1352dd629                     1.0
2808    0xbf118e085855598a04eb12338ba0ebb16525f521                     1.0
2787    0x4c5b8663b34b53052fca9dc6df599c968097b869                     1.0
9184    0xf038925f5817330d93fa79980cd76412191544d4                     1.0
76201   0x4ea8066b2993d27e02ce551cb23dc70e2a2d545c                     1.0
...                                            ...                     ...
48384   0x27d483cc406c87f3059e06381bf81fa456df522a                     0.0
48383   0x179422137be96dc785f00a8d578d3c71d65713ee                     0.0
48381   0x186e740369d5790ce22986711e941f51dcad5090                     0.0
48380   0x186ddd6a828a4914991270904483d3cdb142196e                     0.0
115868  0x447e1351aee219355d88116040fd6124639e51d4                     0.0

[115869 rows x 2 columns]

In [279]:
df_pr.to_sql("page_rank_" + collection, conn, if_exists='append', index=False)
# df_pr.to_sql("page_rank_all", conn, if_exists='append', index=False)

115869

In [280]:
query = "select from_address as target, to_address as source, count(*) as value from \
(select from_address, to_address from (select * from page_rank_" + collection + " limit 100) as a \
join (select * from all_transactions where collection_name = " + collectionWith + ") as b on a.wallet_hash = b.from_address) as c \
join (select * from page_rank_" + collection + " limit 100) as d \
on c.to_address = d.wallet_hash \
group by from_address, to_address \
order by value DESC";
# query = "select from_address as target, to_address as source, count(*) as value from \
# (select from_address, to_address from (select * from page_rank_all limit 100) as a \
# join (select * from all_transactions) as b on a.wallet_hash = b.from_address) as c \
# join (select * from page_rank_all limit 100) as d \
# on c.to_address = d.wallet_hash \
# group by from_address, to_address \
# order by value DESC";
df_pr_top_100_edge = pd.read_sql_query(query, conn)
df_pr_top_100_edge.head()
df_pr_top_100_edge.to_csv('./page_rank_files/pr_top_100_' + collection + '_edge.csv', index=False)
# df_pr_top_100_edge.to_csv('./page_rank_files/pr_top_100_all_edge.csv', index=False)

In [282]:
df_pr_top_100 = pd.read_sql_query("SELECT * from page_rank_" + collection + " limit 100", conn)
df_pr_top_100.head()
df_pr_top_100.to_csv("./page_rank_files/pr_top_100_" + collection + ".csv", index=False)

In [32]:
# df_all_transactions.to_csv('all_transactions.csv', index=False)